In [1]:
from decode_logs import *
import os 

In [2]:
output_dir = "tmp/" + get_current_time()
strategy_type = "kalman"
execution_command = "python backtest_pair.py --strategy_type {} --output_dir {}".format(strategy_type, output_dir)

os.system("rm -rf tmp")
os.system(execution_command)

0

In [3]:
df1, df2 = Decoder.get_strategy_status(output_dir)
dic = Decoder.get_strategy_performance(output_dir)

In [4]:
df1.head(5)

,allow_trade,alpha,buy_amt,buy_stk,data0,data1,date,enter_threshold_size,exit_threshold_size,initial_cash,...,portfolio_value,qty0,qty1,sell_amt,sell_stk,spread,spread_std,status,up_medium,upper_limit
0,False,0.825526,NaN,None,40.275,51.795,2018-05-01,2.0,0.5,NaN,...,1000000.0,0,0,NaN,None,-0.003483,0.046979,0,0.023489,0.093958
1,True,0.825526,NaN,None,40.815,52.345,2018-05-02,2.0,0.5,NaN,...,1000000.0,0,0,NaN,None,0.001116,0.046979,0,0.023489,0.093958
2,True,0.829269,NaN,None,40.485,53.625,2018-05-03,2.0,0.5,NaN,...,1000000.0,0,0,NaN,None,-0.027802,0.046979,0,0.023489,0.093958
3,True,0.628990,NaN,None,41.555,54.165,2018-05-04,2.0,0.5,NaN,...,1000000.0,0,0,NaN,None,0.012612,0.046979,0,0.023489,0.093958
4,True,0.690333,NaN,None,40.365,54.155,2018-05-07,2.0,0.5,NaN,...,1000000.0,0,0,NaN,None,-0.025299,0.046979,0,0.023489,0.093958


In [5]:
df2.head(10)

,index,spread,date,latest_trade_action,sell_stk,buy_stk,buy_amt,sell_amt
0,60,0.101615,2018-07-26,short_spread,AAN,AER,23650.0,27300.0
1,61,-0.016250,2018-07-27,exit_spread,None,None,NaN,NaN
2,62,-0.118242,2018-07-30,long_spread,AER,AAN,34242.0,25654.0
3,69,-0.021204,2018-08-08,exit_spread,None,None,NaN,NaN
4,124,-0.098352,2018-10-25,long_spread,AER,AAN,34578.0,30518.0
5,130,-0.011067,2018-11-02,exit_spread,None,None,NaN,NaN


In [7]:
dic

{'avg_holding_period': 4.666666666666667,
 'endcash': 1177419.7167409933,
 'n_trades': 3,
 'pair': 'AAN-AER',
 'profit': 0.17741971674099333,
 'returnstd': 7361.272303626742,
 'sharperatio': None,
 'startcash': 1000000}